## 通过 Qwen Agent 操作 Redis

参考：[redis](https://github.com/modelcontextprotocol/servers/tree/main/src/redis)

### 一、安装 Redis

在 Ubuntu 安装 Redis：

```bash
# 更新软件包索引
sudo apt update

# 安装 Redis 服务器
sudo apt install redis-server -y

# 启动 Redis 服务
sudo systemctl start redis-server

# 设置开机自启
sudo systemctl enable redis-server
```

调试 `redis-cli`：

```bash
# 新增 key value
SET key value

# 获取 key 对应的 value
GET key

# 删除 key value
DEL key1 key2 ...

# 查找以 "user:" 开头的 key
KEYS user:*
```

## 二、添加样例数据

```bash
# 输入样例数据
SET user:001 shiori
SET user:002 nagisa
SET user:003 harei

# 获取样例数据
GET user:002
```

## 三、Python 连接 Redis

In [1]:
# !uv pip install redis

In [2]:
import redis

# 创建 Redis 连接对象
r = redis.Redis(
    host='localhost',    # Redis 服务器地址
    port=6379,           # 默认端口
    db=0,                # 数据库编号（0-15）
    password=None,       # 如果设置了密码
    decode_responses=True  # 自动将字节解码为字符串
)

# 测试连接
print(r.ping())

True


In [3]:
r.set('user:004', 'mizuki') # 写入
r.setex('user:005', 3600, 'melody')  # 设置过期时间（秒）

True

In [4]:
r.get('user:004')

'mizuki'

In [5]:
r.get('user:005')

'melody'

In [13]:
# 查找以 "user:" 开头的 key
r.keys('user:*')

['user:003', 'user:005', 'user:002', 'user:004', 'user:001']

## 四、MCP 调用 Redis

运行代码前，需要确保 LLM 和 Redis 正常运行。

要启动 LLM 服务，来到项目根目录运行：

```bash
cd test_qwen3
bash vllm_server.sh
```

In [15]:
import os
import asyncio
from typing import Optional

from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI


# 在 jupyter 中运行
ROOT_RESOURCE = os.path.join(os.getcwd(), 'resource')

# 在 python 文件中运行
# ROOT_RESOURCE = os.path.join(os.path.dirname(__file__), 'resource')


def init_agent_service():
    llm_cfg = {
        'model': 'Qwen3-0.6B-FP8',
        'model_server': 'http://localhost:8000/v1',
        'api_key': 'token-kcgyrk',
        'generate_cfg': {
            'top_p': 0.95,
            'temperature': 0.6,
        }
    }
    system = ('你扮演一个 Redis 操作助手，你具有操作 Redis 的能力')
    tools = [{
      "mcpServers": {
        "redis": {
          "command": "npx",
          "args": [
            "-y",
            "@modelcontextprotocol/server-redis",
            "redis://localhost:6379"
          ]
        }
      }
    }]
    bot = Assistant(
        llm=llm_cfg,
        name='Redis 操作助手',
        description='操作 Redis',
        system_message=system,
        function_list=tools,
    )

    return bot

In [16]:
# 初始化 Agent
bot = init_agent_service()

2025-05-21 23:45:57,329 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['redis']
2025-05-21 23:45:57,336 - mcp_manager.py - 245 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: redis


查询 Redis

In [17]:
query = '查询user:001的value'
messages = [{'role': 'user', 'content': query}]

# 输出
response = bot.run_nonstream(messages)
print('bot response:', response)

bot response: [{'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户让我查询user:001的value。首先，我需要确定用户的需求是什么。用户可能是在使用Redis来存储用户信息，现在需要获取某个用户的值。根据提供的工具列表，有一个redis-get函数，它的参数只需要key，所以应该调用这个函数。\n\n接下来，我需要检查是否需要处理可能的参数错误。比如，用户提供的key是否正确，是否符合Redis的语法。这里的key是user:001，看起来没问题。所以直接调用redis-get，参数是key: "user:001"。\n\n有没有可能用户希望存储这个信息？不过用户只是查询，所以不需要设置或删除。确认所有参数都正确后，生成对应的tool_call。\n', 'name': 'Redis 操作助手'}, {'role': 'assistant', 'content': '', 'name': 'Redis 操作助手', 'function_call': {'name': 'redis-get', 'arguments': '{"key": "user:001"}'}}, {'role': 'function', 'content': 'shiori', 'name': 'redis-get'}, {'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户之前让我查询user:001的value，我执行了redis-get命令，返回了shiori。现在用户可能需要确认这个结果是否正确，或者有其他需求。我应该回复用户，说明查询的结果，并询问是否有其他问题。保持友好和帮助的态度，避免让用户感到困惑。\n', 'name': 'Redis 操作助手'}, {'role': 'assistant', 'content': '\n\n查询结果为：shiori。需要帮助吗？', 'name': 'Redis 操作助手'}]


In [18]:
print('result:', response[-1]['content'].strip())

result: 查询结果为：shiori。需要帮助吗？


新增 Redis 记录

In [19]:
query = '增加一条记录，key为user:006，value为yukisyo'
messages = [{'role': 'user', 'content': query}]

# 输出
response = bot.run_nonstream(messages)
print('bot response:', response)

bot response: [{'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户让我增加一条记录，key是user:006，value是yukisyo。首先，我需要确认用户的需求是否正确。他们提供的参数看起来符合Redis的基本操作，key和value都是字符串，没有提到expiration时间，所以可能不需要设置过期时间。\n\n接下来，我需要检查可用的工具。用户提供的工具中有redis-set，这个函数正好用于设置键值对，支持可选的过期时间。参数需要key和value，而用户已经提供了这两个参数，所以直接调用redis-set即可。\n\n然后，我需要确保参数的格式正确。key是字符串，value也是字符串，符合要求。没有其他参数需要处理，所以直接构造JSON参数调用即可。不需要使用其他工具，因为用户只需要设置一个简单的记录，不需要获取或删除。\n\n最后，生成对应的tool_call，确保JSON格式正确，参数正确无误。这样用户的需求就能被准确处理了。\n', 'name': 'Redis 操作助手'}, {'role': 'assistant', 'content': '', 'name': 'Redis 操作助手', 'function_call': {'name': 'redis-set', 'arguments': '{"key": "user:006", "value": "yukisyo"}'}}, {'role': 'function', 'content': 'Successfully set key: user:006', 'name': 'redis-set'}, {'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户之前让我增加一条记录，key是user:006，value是yukisyo。我使用了redis-set函数，参数正确，返回成功。现在用户可能希望确认这个操作是否成功，或者需要进一步操作。不过用户没有给出新的问题，所以应该结束对话。需要检查是否有遗漏，但根据之前的交互，用户似乎只是测试或完成任务。因此，回复应该简洁，确认成功，并询问是否需要帮助。\n', 'name': 

In [20]:
print('result:', response[-1]['content'].strip())

result: 成功！Redis中key `user:006` 已更新为 `yukisyo`。您还有其他需求吗？


In [21]:
r.get('user:006')

'yukisyo'